# Baseball States GPT-2 Training on Google Colab

This notebook trains a GPT-2 model to predict baseball game states.

## Setup
1. Mount Google Drive
2. Clone the repository
3. Load config and override paths for Colab environment
4. Train model

In [3]:
# Configuration
GITHUB_USERNAME = 'angerami'
REPO_NAME = 'baseball-states'
CONFIG_FILE = 'configs/colab_config.yaml'  # Relative to repo root
RUN_NUMBER = 'run-001'
GDRIVE_PATH = '/Users/angerami/Google Drive/My Drive'

In [4]:
import os
import sys

def get_colab_environment():
    # 1. Check if we are even in a Colab-managed runtime
    if 'google.colab' not in sys.modules:
        return "Local/Other Jupyter"

    # 2. Check for VS Code specific environment variables
    if os.environ.get('TERM_PROGRAM') == 'vscode' or 'VSCODE_PID' in os.environ:
        return "VS Code (Colab Extension)"

    # 3. Check for the browser-exclusive 'COLAB_RELEASE_TAG'
    if 'COLAB_RELEASE_TAG' in os.environ:
        return "Browser (colab.research.google.com)"

    return "Unknown Colab Environment"

env = get_colab_environment()
print(f"Running in: {env}")


drive_available = (env == "Browser (colab.research.google.com)")
print(f"Google Drive mounting supported: {drive_available}")
if drive_available:
    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive')
    GDRIVE_PATH = '/content/drive/MyDrive'

SECRETS_PATH = f'{GDRIVE_PATH}/secrets/colab-github'
DATA_PATH = f'{GDRIVE_PATH}/Data/{REPO_NAME}'
    # Load GitHub token
with open(SECRETS_PATH) as f:
    github_token = f.read().strip()

Running in: Browser (colab.research.google.com)
Google Drive mounting supported: True
Mounted at /content/drive


In [6]:
# Clone repository (or pull if it exists)
if not os.path.exists(REPO_NAME):
    !git clone https://{github_token}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git
else:
    print(f"Repository {REPO_NAME} already exists, pulling latest changes...")
    !cd {REPO_NAME} && git pull

# Add repo to Python path
import sys
sys.path.insert(0, f'/content/{REPO_NAME}')

Repository baseball-states already exists, pulling latest changes...
Already up to date.


In [8]:
# Install dependencies if needed
!pip install -e $REPO_NAME

Obtaining file:///content/baseball-states
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 136.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 144.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.2 MB/s eta 0:00:00
  Running setup.py develop for baseball-states


In [10]:
from baseball_states.training import ModelConfig, train_model

# Load config from file
config = ModelConfig.from_file(f'{REPO_NAME}/{CONFIG_FILE}')

# Override paths for this specific run (config serves as template)
config.data_path = f"{DATA_PATH}/data/tokens_inning"
config.output_dir = f"{DATA_PATH}/checkpoints/{RUN_NUMBER}"

# Optional: Override other parameters for this run
# config.num_epochs = 3
# config.batch_size = 32

# Display the config
print("Training configuration:")
print(config)
print()

# Train the model
model, tokenizer = train_model(config)

# Save the actual config used for this run
config.to_file(f"{config.output_dir}/config_used.yaml")

Training configuration:
ModelConfig(
  n_embd=64,
  n_layer=6,
  n_head=4,
  n_positions=32,
  batch_size=64,
  num_epochs=1,
  learning_rate=0.0005,
  warmup_steps=10,
  weight_decay=0.01,
  data_path='/content/drive/MyDrive/Data/baseball-states/data/tokens_inning',
  val_split=0.1,
  max_length=32,
  train_fraction=1.0,
  use_packing=False,
  output_dir='/content/drive/MyDrive/Data/baseball-states/checkpoints/run-001',
  save_steps=500,
  eval_steps=100,
  logging_steps=10,
  save_initial_checkpoint=True,
  resume_from_checkpoint='auto',
  seed=42,
)

Using device: cuda
Train samples: 343975
Eval samples: 38220
Model parameters: 303,936
Saving initial checkpoint to /content/drive/MyDrive/Data/baseball-states/checkpoints/run-001/checkpoint-initial
Initial checkpoint saved!
Run name: gpt2-train-2025-1223-221255
No checkpoint found, starting training from scratch
Starting training...

Training started at 2025-12-23 22:12:56



`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss,Accuracy,Top3 Accuracy,Top5 Accuracy,Perplexity,Illegal Prob,Runs Scored
100,1.148800,1.138857,0.190224,0.696579,0.773959,14.069635,0.086288,27.856228
200,1.116400,1.077359,0.190180,0.690145,0.837054,14.642381,0.089302,26.915541
300,1.059800,1.067977,0.190219,0.695509,0.800366,15.869293,0.090578,26.910807
400,1.064400,1.057088,0.190180,0.687219,0.842189,14.377877,0.090248,27.905704
500,1.053100,1.054341,0.190180,0.687199,0.830043,13.407660,0.091158,26.948011
600,1.132200,1.052911,0.190180,0.690468,0.829505,12.931553,0.091988,27.905312
700,1.055300,1.046559,0.190180,0.681024,0.834785,14.454954,0.091228,27.110701
800,0.979500,1.053356,0.190185,0.673008,0.830281,15.054455,0.090820,27.591993
900,1.052100,1.047457,0.190180,0.708028,0.832904,14.704562,0.089950,27.905861
1000,1.061700,1.041401,0.190180,0.686925,0.835218,14.262701,0.090571,27.989195



[Step 10/5375 (0.2%)] Elapsed: 0:00:01 | ETA: 0:02:57

[Step 20/5375 (0.4%)] Elapsed: 0:00:01 | ETA: 0:03:05

[Step 30/5375 (0.6%)] Elapsed: 0:00:01 | ETA: 0:02:56

[Step 40/5375 (0.7%)] Elapsed: 0:00:02 | ETA: 0:02:51

[Step 50/5375 (0.9%)] Elapsed: 0:00:02 | ETA: 0:02:50

[Step 60/5375 (1.1%)] Elapsed: 0:00:02 | ETA: 0:02:44

[Step 70/5375 (1.3%)] Elapsed: 0:00:03 | ETA: 0:02:39

[Step 80/5375 (1.5%)] Elapsed: 0:00:03 | ETA: 0:02:41

[Step 90/5375 (1.7%)] Elapsed: 0:00:03 | ETA: 0:02:40

[Step 100/5375 (1.9%)] Elapsed: 0:00:04 | ETA: 0:02:37

[Step 100/5375 (1.9%)] Elapsed: 0:00:32 | ETA: 0:02:37

[Step 110/5375 (2.0%)] Elapsed: 0:00:33 | ETA: 0:54:23

[Step 120/5375 (2.2%)] Elapsed: 0:00:33 | ETA: 0:54:19

[Step 130/5375 (2.4%)] Elapsed: 0:00:33 | ETA: 0:54:15

[Step 140/5375 (2.6%)] Elapsed: 0:00:34 | ETA: 0:54:12

[Step 150/5375 (2.8%)] Elapsed: 0:00:34 | ETA: 0:02:48

[Step 160/5375 (3.0%)] Elapsed: 0:00:34 | ETA: 0:02:48

[Step 170/5375 (3.2%)] Elapsed: 0:00:35 | ETA: 0:03:01



There were missing keys in the checkpoint model loaded: ['lm_head.weight'].



[Step 5375/5375 (100.0%)] Elapsed: 0:28:49 | ETA: 0:00:00

Training completed!
Total time: 0:28:49
End time: 2025-12-23 22:41:46

Saving model to /content/drive/MyDrive/Data/baseball-states/checkpoints/run-001
Running final evaluation...



[Step 5375/5375 (100.0%)] Elapsed: 0:29:18 | ETA: 0:00:00
Final eval loss: 1.0307
Final perplexity: 2.8030
Final accuracy: 0.1902
Config saved to /content/drive/MyDrive/Data/baseball-states/checkpoints/run-001/config_used.yaml


In [ ]:
# Optional: View training in TensorBoard
%load_ext tensorboard
%tensorboard --logdir {config.output_dir}/logs

In [ ]:
# Optional: Quick inference test
import torch

# Generate a sample prediction
model.eval()
with torch.no_grad():
    # Create a simple test input (modify as needed)
    test_input = tokenizer.encode("<BOS>", return_tensors="pt")
    output = model.generate(test_input, max_length=20, num_return_sequences=1)
    print("Sample generation:")
    print(tokenizer.decode(output[0]))